In [1]:
#!pip install tensorflow==2.5.0

In [2]:
# this is used in downloading data from the google drive
#!pip install gdown

Below is code with a link to a happy or sad dataset which contains 80 images, 40 happy and 40 sad. 
Create a convolutional neural network that trains to 100% accuracy on these images,  which cancels training upon hitting training accuracy of >.999

Hint -- it will work best with 3 convolutional layers.

In [3]:
import tensorflow as tf
import os
import zipfile

!gdown --id 1NvV6VhmrfU8JDZNoEbwJxwx_6dhyN5bf 

zip_ref = zipfile.ZipFile("./happy-or-sad.zip", 'r')
zip_ref.extractall("./h-or-s")
zip_ref.close()

Downloading...
From: https://drive.google.com/uc?id=1NvV6VhmrfU8JDZNoEbwJxwx_6dhyN5bf
To: /content/happy-or-sad.zip
100% 2.67M/2.67M [00:00<00:00, 84.4MB/s]


In [6]:
def train_happy_sad_model():

    DESIRED_ACCURACY = 0.95

    class myCallback(tf.keras.callbacks.Callback): 
        def on_epoch_end(self, epoch, logs={}):
            if logs.get('accuracy') is not None and logs.get('accuracy') > DESIRED_ACCURACY:
                print("\nReached 95% accuracy so cancelling training!")
                self.model.stop_training = True
        
    callbacks = myCallback()

    model = tf.keras.models.Sequential([
    tf.keras.layers.Conv2D(16, (3,3), activation='relu', input_shape=(150, 150, 3)), 
    tf.keras.layers.MaxPooling2D(2, 2), 
    tf.keras.layers.Conv2D(32, (3,3), activation='relu'), 
    tf.keras.layers.MaxPooling2D(2,2), 
    tf.keras.layers.Conv2D(32, (3,3), activation='relu'),  
    tf.keras.layers.MaxPooling2D(2,2),  
    tf.keras.layers.Flatten(),  
    tf.keras.layers.Dense(512, activation='relu'),  
    tf.keras.layers.Dense(1, activation='sigmoid')  
    ])

    from tensorflow.keras.optimizers import RMSprop

    model.compile(loss='binary_crossentropy', 
                  optimizer=RMSprop(learning_rate=0.001), 
                  metrics=['accuracy'])
    
    # This code block should create an instance of an ImageDataGenerator called train_datagen 
    # And a train_generator by calling train_datagen.flow_from_directory

    from tensorflow.keras.preprocessing.image import ImageDataGenerator

    train_datagen = ImageDataGenerator(rescale=1/255)  

    # Please use a target_size of 150 X 150.
    train_generator = train_datagen.flow_from_directory( "./h-or-s",  
                                                        target_size=(150, 150),  
                                                        batch_size=10,  
                                                        class_mode='binary')  

    history = model.fit(train_generator, 
                        steps_per_epoch=8, 
                        epochs=15, 
                        verbose=1, 
                        callbacks=[callbacks] 
                       )
    
    return history.history['accuracy'][-1]

In [7]:
train_happy_sad_model()

Found 80 images belonging to 2 classes.
Epoch 1/15
8/8 [==============================] - 1s 27ms/step - loss: 1.9170 - accuracy: 0.5000
Epoch 2/15
8/8 [==============================] - 0s 29ms/step - loss: 0.6293 - accuracy: 0.6500
Epoch 3/15
8/8 [==============================] - 0s 28ms/step - loss: 0.5015 - accuracy: 0.7375
Epoch 4/15
8/8 [==============================] - 0s 28ms/step - loss: 0.2603 - accuracy: 0.9125
Epoch 5/15
7/8 [=========================>....] - ETA: 0s - loss: 0.1176 - accuracy: 0.9714
Reached 95% accuracy so cancelling training!
8/8 [==============================] - 0s 26ms/step - loss: 0.1051 - accuracy: 0.9750


0.9750000238418579